In [125]:
# import libraries
import re
import os
import json
import uuid
import time
import datetime
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

In [126]:
# Load environment variables from .env file
load_dotenv()

# Get API key from environment variables
open_ai_key = os.getenv("SBS_OPENAI_API_KEY")

# Initialize OpenAI client
client = OpenAI()
client.api_key = open_ai_key

JSONL_FILE = "requests.jsonl"
OUTPUT_FILE = "output.jsonl"
VOCAB_FOLDER_PATH = "pali_class/vocab"
EXERCISE_FOLDER_PATH = "pali_class/exercises"

In [127]:
new_suttas = pd.read_csv("pali_class/new_suttas.csv")

In [128]:
# keyword = input("Enter a Pali word: ").strip() # Remove extra spaces

keyword = "anuttara"

def search_pali_in_csv(keyword):
    """Search for a Pali word in all CSV files in the folder."""
    result = {
        "id": -1,
        "pali": "",
        "meaning": "",
        "pos": "",
        "exercise_number": "",
        "prdc": []
    }

    for filename in os.listdir(VOCAB_FOLDER_PATH):
        if filename.endswith(".csv"): # Only search in CSV files
            file_path = os.path.join(VOCAB_FOLDER_PATH, filename)
            df = pd.read_csv(file_path, dtype=str)
            example_columns = [col for col in df.columns if "example" in col.lower()]
            match = df[df["pali"] == keyword]

            if not match.empty:
                result["id"] = match["id"].values[0]
                result["pali"] = match["pali"].values[0]
                result["meaning"] = match["meaning"].values[0]
                result["pos"] = match["pos"].values[0]

                for col in example_columns:
                    sentence = match[col].values[0]
                    if isinstance(sentence, str): # Ensure it's a string before regex
                        # Extract text inside <b>...</b>
                        extracted = re.findall(r"<b>(.*?)</b>", sentence)
                        
                        for ext in extracted:
                            if ext not in result['prdc']:
                                result['prdc'].append(ext)
                    
                print("Match found in file:", filename)
                
                number = filename.split("_")[-1].split(".")[0] # Extract the number
                result["exercise_number"] = number

                break

    if result["id"] == -1:
        print("No matches found")

    return result

result = search_pali_in_csv(keyword)
result

Match found in file: vocab_class_2.csv


{'id': '4524',
 'pali': 'anuttara',
 'meaning': 'highest (of); unsurpassed (by); incomparable (to); superior (to)',
 'pos': 'adj',
 'exercise_number': '2',
 'prdc': ['anuttaro', 'anuttaraṃ']}

In [129]:
# Find exercise number
exercise_number = result['exercise_number']
found_exercise = False
target_exercise = ""
exercise_data = ""

for filename in os.listdir(EXERCISE_FOLDER_PATH):
    if filename.endswith(".txt") and f"_{exercise_number}." in filename:
        found_exercise = True
        target_exercise = filename
        print("Found:", filename)

if not found_exercise:
    print("Exercise not found")
else:
    exercise_data = open(os.path.join(EXERCISE_FOLDER_PATH, target_exercise), "r").read().strip()
    print(exercise_data)

Found: exercises_class_2.txt
Class 2 Exercises

namo tassa bhagavato arahato sammā-sambuddhassa	 					
Homage to him, the Blessed One, the Worthy One, the fully Enlightened One.
 
avijjāya tv'eva asesa-virāga-nirodhā saṅkhāra-nirodho, saṅkhāra-nirodhā viññāṇa-nirodho, viññāṇa-nirodhā nāmarūpa-nirodho, nāmarūpa-nirodhā saḷāyatana-nirodho, saḷāyatana-nirodhā phassa-nirodho, phassa-nirodhā vedanā-nirodho, vedanā-nirodhā taṇhā-nirodho, taṇhā-nirodhā upādāna-nirodho, upādāna-nirodhā bhava-nirodho, bhava-nirodhā jāti-nirodho, jāti-nirodhā jarā-maraṇaṃ soka-parideva-dukkha-domanass'upāyāsā nirujjhanti. 
But from the complete fading away and cessation of ignorance there is cessation of volitional formations; from the cessation of volitional formations, cessation of consciousness; from the cessation of consciousness, cessation of name-and-form; from the cessation of name-and-form, cessation of the six sense bases; from the cessation of the six sense bases, cessation of contact; from the cessati

In [155]:
SYSTEM_PROMPT = """
You are an AI assistant specialized in Pali language processing. Your primary task is to extract **full sentences** from an exercise dataset based on a given Pali word. Follow these instructions carefully:

1. **Extract and Structure Data:**
   - Identify sentences in the exercise dataset that contain the **given Pali word**, recognizing its various forms (e.g., declensions and conjugations).
   - Extract **full sentences** that provide meaningful context. Do **not** return single-word outputs (e.g., "<b>Anussarati</b>").
   - A valid **sutta reference number** (also known as the **sutta number**) must be **explicitly present within the selected sentence**.
   - **STRICT RULE**: 
      - If the **sutta reference number** (e.g., "AN 3.71") is **not found directly in the selected sentence itself**, **discard the selected sentence**.
   - Remove any **sutta reference number** from the selected sentence (e.g., "AN 3.71", "AN 3.71 (simpl)", or similar citation formats) once identified.
   - Store the **sutta reference number** in the `"class_source"` field.
   - **Handling `(simpl)` in `class_source` Formatting:**
      - If the extracted **sutta reference number** contains **"(simpl)"**, preserve it (e.g., `"DN 19.7 (simpl)"`).
      - If **"(simpl)"** is **not present**, return only the core sutta number (e.g., `"DN 19.7"`).

2. **Ensure Correct Sutta Reference Name Retrieval:**
   - The **sutta reference name** (also known as the **sutta reference name**) will not be available in the exercise dataset.
   - After obtaining the **sutta reference number**, retrieve the corresponding **sutta reference name**.
      - Example: If **sutta reference number** is `"AN 2.1"`, the **sutta reference name** should be `"vajjasuttaṃ"`.
      - If the correct **sutta reference name** cannot be determined, return `"unknown"` instead of making an incorrect guess.
   - Store the **sutta reference name** in the `"class_sutta"` field.

3. **Formatting Rule for `"class_example"`:**
   - The **selected Pali word** is **highlighted alone** within `<b>...</b>`.
   - The capitalization of the entire sentence must remain exactly as in the exercise dataset.

4. **`"pali"` Field Rule:**  
   - The `"pali"` field **must always** match the **exact given Pali word**, including numbering if present (e.g., `"anussarati 1.1"`).
   - Even if the extracted sentence contains a different **morphological form**, the `"pali"` field must **not** be altered.

5. **`"explanation"` Field Rule:**  
   - The `"explanation"` field should provide a brief justification for why the response was generated **if the user asks a question**.
   - If the user requests a solution, provide a practical suggestion to achieve the desired outcome.
   - If **no question is provided**, return an **empty string** (`""`).

6. **Output Structure and Formatting:**
   - If a valid sentence is found and the sutta reference number is explicitly present within it, return the output in **JSON format** with the following structure:
      {
         "id": "<Given Pali ID>",
         "pali": "<Given Pali word>",
         "selected_pali_word": "<Selected Pali word>",
         "class_source": "<Sutta Reference Number>",
         "class_sutta": "<Sutta Reference Name>",
         "class_example": "<A full sentence with <b>Selected Pali word</b> highlighted, preserving original capitalization>",
         "english_translation": "<English translation>",
         "explanation": "<A justification for why this response was generated and a solution if applicable, or an empty string if no question was asked>"
      }
   - If the sutta reference number is missing from the selected sentence, return an **empty JSON object** `{}`.
   - If no valid sentence is found, return an **empty JSON object** `{}`.

7. **Strict JSON Output rules:**
   **IMPORTANT: FOLLOW THESE RULES STRICTLY**
   - **DO NOT** wrap the JSON output in Markdown formatting (e.g., ` ```json ... ``` `).
   - **DO NOT** add any extra text before or after the JSON response.
   - **DO NOT** format the output as a code block.
   - **ONLY RETURN A PLAIN JSON OBJECT.**
   - **Failure to follow these rules will result in an invalid response.**
"""

In [156]:
# if len(result["prdc"]) > 0:
#     USER_PROMPT = f"""
# For the Pali word: "{result['pali']}" and its id: "{result['id']}", \
# which has "{result['pos']}" as its grammatical part of speech, "{result['meaning']}" as Pali word`s english meaning and \
# **Found declensions and conjugations**: {', '.join(result['prdc'])}, \
# find example sentence in exercise data: "{exercise_data}".
# """
# else:
#     USER_PROMPT = f"""
# For the Pali word: "{result['pali']}" and its id: "{result['id']}", \
# which has "{result['pos']}" as its grammatical part of speech and "{result['meaning']}" as Pali word`s english meaning, \
# find example sentence in exercise data: "{exercise_data}".
# """

USER_PROMPT = f"""
For the given Pali word: "{result['pali']}" with ID: "{result['id']}", \
extract a relevant sentence in exercise data: "{exercise_data}". \
User question: Why this sentence is relevant as there are not any sutta reference number in it?
"""

print(USER_PROMPT)


For the given Pali word: "anuttara" with ID: "4524", extract a relevant sentence in exercise data: "Class 2 Exercises

namo tassa bhagavato arahato sammā-sambuddhassa	 					
Homage to him, the Blessed One, the Worthy One, the fully Enlightened One.
 
avijjāya tv'eva asesa-virāga-nirodhā saṅkhāra-nirodho, saṅkhāra-nirodhā viññāṇa-nirodho, viññāṇa-nirodhā nāmarūpa-nirodho, nāmarūpa-nirodhā saḷāyatana-nirodho, saḷāyatana-nirodhā phassa-nirodho, phassa-nirodhā vedanā-nirodho, vedanā-nirodhā taṇhā-nirodho, taṇhā-nirodhā upādāna-nirodho, upādāna-nirodhā bhava-nirodho, bhava-nirodhā jāti-nirodho, jāti-nirodhā jarā-maraṇaṃ soka-parideva-dukkha-domanass'upāyāsā nirujjhanti. 
But from the complete fading away and cessation of ignorance there is cessation of volitional formations; from the cessation of volitional formations, cessation of consciousness; from the cessation of consciousness, cessation of name-and-form; from the cessation of name-and-form, cessation of the six sense bases; from the 

In [158]:
llm_model = ["gpt-4o", "gpt-4-turbo"]

# Define single request
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system", 
            "content": SYSTEM_PROMPT
        },
        {
            "role": "user", 
            "content": USER_PROMPT
        }
    ],
    max_tokens=1000
)

# Extract response content
response_text = response.choices[0].message.content

# Convert string to JSON
try:
    response_json = json.loads(response_text)
    print(json.dumps(response_json, indent=4, ensure_ascii=False)) # Pretty print
    # sutta_name = new_suttas[new_suttas['sutta_number'] == response_json['class_source'].replace(" ", "").replace("(simpl)", "")]["sutta_name"].tolist()
    # print(sutta_name)
except json.JSONDecodeError:
    print("Invalid JSON response:", response_text)

{}


In [76]:
eval_vocab_df = pd.read_csv("pali_class/vocab/vocab_class_2.csv")
pick_palis = eval_vocab_df["pali"]
pick_palis

0               attha 2.1
1               anālaya 1
2                anuttara
3                 apara 1
4      aparena samayena 1
              ...        
118            vūpasama 1
119              vedanā 1
120              hattha 1
121            adhigama 2
122              ābādha 2
Name: pali, Length: 123, dtype: object

In [77]:
eval_exercise_data = open("pali_class/exercises/exercises_class_2.txt", "r").read().strip()
print(eval_exercise_data)

Class 2 Exercises

namo tassa bhagavato arahato sammā-sambuddhassa	 					
Homage to him, the Blessed One, the Worthy One, the fully Enlightened One.
 
avijjāya tv'eva asesa-virāga-nirodhā saṅkhāra-nirodho, saṅkhāra-nirodhā viññāṇa-nirodho, viññāṇa-nirodhā nāmarūpa-nirodho, nāmarūpa-nirodhā saḷāyatana-nirodho, saḷāyatana-nirodhā phassa-nirodho, phassa-nirodhā vedanā-nirodho, vedanā-nirodhā taṇhā-nirodho, taṇhā-nirodhā upādāna-nirodho, upādāna-nirodhā bhava-nirodho, bhava-nirodhā jāti-nirodho, jāti-nirodhā jarā-maraṇaṃ soka-parideva-dukkha-domanass'upāyāsā nirujjhanti. 
But from the complete fading away and cessation of ignorance there is cessation of volitional formations; from the cessation of volitional formations, cessation of consciousness; from the cessation of consciousness, cessation of name-and-form; from the cessation of name-and-form, cessation of the six sense bases; from the cessation of the six sense bases, cessation of contact; from the cessation of contact, cessation of f

In [203]:
def generate_unique_id(user_id):
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S") # Format: YYYYMMDDHHMMSS
    unique_str = uuid.uuid4().hex
    return f"{user_id}-{timestamp}-{unique_str}"

for p in pick_palis:    
    result = search_pali_in_csv(p)
    # Generate a unique ID for the request (user-timestamp-unique)
    unique_id = generate_unique_id("user-1")
    
    # if len(result["prdc"]) > 0:
    #     USER_PROMPT = f"""
    # For the Pali word: "{result['pali']}" and its id: "{result['id']}", \
    # which has "{result['pos']}" as its grammatical part of speech, "{result['meaning']}" as Pali word`s english meaning and \
    # **Found declensions and conjugations**: {', '.join(result['prdc'])}, \
    # find example sentence in exercise data: "{exercise_data}".
    # """
    # else:
    #     USER_PROMPT = f"""
    # For the Pali word: "{result['pali']}" and its id: "{result['id']}", \
    # which has "{result['pos']}" as its grammatical part of speech and "{result['meaning']}" as Pali word`s english meaning, \
    # find example sentence in exercise data: "{exercise_data}".
    # """

    USER_PROMPT = f"""
    For the given Pali word: "{result['pali']}" with ID: "{result['id']}", \
    extract a relevant sentence in exercise data: "{exercise_data}". \
    """

    request = {
        "custom_id": unique_id, 
        "method": "POST", 
        "url": "/v1/chat/completions", 
        "body": {
            "model": "gpt-4-turbo", 
            "messages": [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": USER_PROMPT}
            ],
            "max_tokens": 1000
        }
    }

    # Write to JSONL file (append mode for multiple requests)
    with open(JSONL_FILE, "a", encoding="utf-8") as f:
        json.dump(request, f) # Convert dictionary to JSON string
        f.write("\n") # Newline for the next JSON object

    print(f"Request saved to {JSONL_FILE}")

Match found in file: vocab_class_2.csv
Request saved to requests.jsonl
Match found in file: vocab_class_2.csv
Request saved to requests.jsonl
Match found in file: vocab_class_2.csv
Request saved to requests.jsonl
Match found in file: vocab_class_2.csv
Request saved to requests.jsonl
Match found in file: vocab_class_2.csv
Request saved to requests.jsonl
Match found in file: vocab_class_2.csv
Request saved to requests.jsonl
Match found in file: vocab_class_2.csv
Request saved to requests.jsonl
Match found in file: vocab_class_2.csv
Request saved to requests.jsonl
Match found in file: vocab_class_2.csv
Request saved to requests.jsonl
Match found in file: vocab_class_2.csv
Request saved to requests.jsonl
Match found in file: vocab_class_2.csv
Request saved to requests.jsonl
Match found in file: vocab_class_2.csv
Request saved to requests.jsonl
Match found in file: vocab_class_2.csv
Request saved to requests.jsonl
Match found in file: vocab_class_2.csv
Request saved to requests.jsonl
Match 

In [204]:
batch_input_file = client.files.create(
    file=open(JSONL_FILE, "rb"),
    purpose="batch"
)

batch_input_file

FileObject(id='file-WaPd1128PWBVj5jyQ4wmWo', bytes=2017578, created_at=1740831737, filename='requests.jsonl', object='file', purpose='batch', status='processed', status_details=None, expires_at=None)

In [205]:
batch_input_file_id = batch_input_file.id

# Start the batch process and store the response
batch_request = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={"description": "Automated batch processing"}
)

# Get Batch ID
batch_id = batch_request.id
print("Batch ID:", batch_id)

Batch ID: batch_67c2fbfa22a481909f746b3da5f8442b


In [206]:
# Function to check if the batch is completed
def wait_for_batch_completion(client, batch_id, check_interval=5):
    """Waits for the batch job to complete before proceeding."""
    while True:
        batch_status = client.batches.retrieve(batch_id)
        status = getattr(batch_status, "status", "").lower() # Access attribute directly

        print(f"Batch Status: {status}")

        if status in ["completed", "failed", "cancelled"]:
            break # Stop polling when the batch is done

        time.sleep(check_interval) # Wait before checking again

In [207]:
wait_for_batch_completion(client, batch_id)

# Proceed to the next step after completion
print("Batch processing is complete. Proceeding to the next step.")

Batch Status: in_progress
Batch Status: in_progress
Batch Status: in_progress
Batch Status: in_progress
Batch Status: in_progress
Batch Status: in_progress
Batch Status: in_progress
Batch Status: in_progress
Batch Status: in_progress
Batch Status: in_progress
Batch Status: in_progress
Batch Status: in_progress
Batch Status: in_progress
Batch Status: in_progress
Batch Status: in_progress
Batch Status: in_progress
Batch Status: in_progress
Batch Status: in_progress
Batch Status: finalizing
Batch Status: finalizing
Batch Status: finalizing
Batch Status: completed
Batch processing is complete. Proceeding to the next step.


In [208]:
output_file_id = client.batches.retrieve(batch_id).output_file_id
file_response = client.files.content(output_file_id)
print(file_response.text)

{"id": "batch_req_67c2fc5bf7388190bc34c26ba795e719", "custom_id": "user-1-20250301200407-9eb5eacd6bcf46e0814bf7c688558c27", "response": {"status_code": 200, "request_id": "d43d8079c0fec2cc7df5a8e739223132", "body": {"id": "chatcmpl-B6G9DGB2wK4aPGf0nXRCUhXxLmLw7", "object": "chat.completion", "created": 1740831739, "model": "gpt-4-turbo-2024-04-09", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\n    \"id\": \"2603\",\n    \"pali\": \"attha 2.1\",\n    \"selected_pali_word\": \"attho\",\n    \"class_source\": \"SN 22.1 (simpl)\",\n    \"class_sutta\": \"unknown\",\n    \"class_example\": \"It would be good if the meaning of this statement may become evident to Venerable S\u0101riputta. (so that he could explain it to us)\",\n    \"english_translation\": \"It would be good if the meaning of this statement may become evident to Venerable S\u0101riputta. (so that he could explain it to us)\",\n    \"explanation\": \"\"\n}", "refusal": null}, "logprobs": null, "fini

In [209]:
# Initialize an empty DataFrame with predefined columns
df = pd.DataFrame(columns=["id", "pali", "class_source", "class_sutta", "class_example", "english_translation"])

for line in file_response.iter_lines():
    if line: # Ignore empty lines
        try:
            data = json.loads(line) # Parse each JSON object
        
            # Extract response text from OpenAI output
            response_text = data.get("response", {}).get("body", {}).get('choices', [{}])[0].get('message', {}).get('content', '{}')

            # Convert response_text (JSON string) into a dictionary
            extracted_data = json.loads(response_text)

            df = pd.concat([df, pd.DataFrame([extracted_data])], ignore_index=True)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}, Line: {line}")

print(df)

Error decoding JSON: Expecting value: line 1 column 1 (char 0), Line: {"id": "batch_req_67c2fc62f5c4819093da942212f09a3f", "custom_id": "user-1-20250301202112-5167e03d48f844f39bf498044af9751d", "response": {"status_code": 200, "request_id": "8c3c18ead2dfd981cfad4ec9dba09cbe", "body": {"id": "chatcmpl-B6G9VOtib3m8zU28E8Ubuv3ucmhcg", "object": "chat.completion", "created": 1740831757, "model": "gpt-4-turbo-2024-04-09", "choices": [{"index": 0, "message": {"role": "assistant", "content": "```json\n{}\n```", "refusal": null}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 2234, "completion_tokens": 6, "total_tokens": 2240, "prompt_tokens_details": {"cached_tokens": 0, "audio_tokens": 0}, "completion_tokens_details": {"reasoning_tokens": 0, "audio_tokens": 0, "accepted_prediction_tokens": 0, "rejected_prediction_tokens": 0}}, "service_tier": "default", "system_fingerprint": "fp_bf9cb2c77f"}}, "error": null}
Error decoding JSON: Expecting value: line 1 column 1 (char

In [210]:
# Save DataFrame to CSV
df.to_csv("pali_class/output/output-2.csv", index=False, encoding="utf-8")

print("CSV file saved as pali_class/output/output-2.csv")

CSV file saved as pali_class/output/output-2.csv
